## **空格觀點的 puzzle-based reshuffling**

### **集合**
---
$$
\begin{array}{ll}
   T & \text{時間集合，$T = \{1,2,3,...,|T|\}$}  \\
   N & \text{網格格點集合}  \\
   A & \text{格點間路徑集合}  \\
   R & \text{各類推盤集合，$R = \{0,1,2,...,|R|-1\}$}  \\
   \delta^{in}_{ri} & \text{推盤 $r\in R$ 流入格點 $i \in N$ 之節線集合， $\delta^{in}_{ri} \subset A$ } \\
   \delta^{out}_{ri} & \text{推盤 $r\in R$ 流出格點 $i \in N$ 之節線集合， $\delta^{out}_{ri} \subset A$ } \\
   P_{ij} & \text{虛擬推盤自格點 $i \in N$ 到格點 $j \in N$ 路徑中經過格點之集合， $P_{ij} \subset N$} \\
   C_i & \text{虛擬推盤移動路徑中經過格點 $i \in N$ 之節線集合， $C_i \subset A$} \\
   V_i & \text{虛擬推盤垂直進出格點 $i \in N$ 之節線集合， $V_i \subset A$ } \\
   H_i & \text{虛擬推盤水平進出格點 $i \in N$ 之節線集合， $H_i \subset A$ } \\
\end{array}\\
$$

### **參數**
---

$$
\begin{array}{ll}
   M & \text{極大懲罰數} \\
   \epsilon & \text{極小的數} \\
   D & \text{虛擬迄點} \\
   S_i^r & \text{推盤 $r\in R$ 初始位置在格點 $i \in N$ 為1；反之為0} \\
   F_r & \text{出貨推盤 $r\in R \backslash \{0\}$ 的終點位置，$F_r \in N$} \\
   Num^r & \text{各類推盤 $r \in R$ 數量} \\
   |T| & \text{揀貨時間上限} \\
   |P_{ij}| & \text{虛擬推盤自格點 $i \in N$ 到格點 $j \in N$ 之路徑長度}
\end{array}\\
$$

### **變數**
---
$$
\begin{array}{ll}
   f^{tr}_{ij} & \text{推盤 $r\in R$ 在時刻 $t \in T$ 自格點 $i \in N$ 移至格點 $j \in N$ 為1；反之為0}  \\
   w & \text{出貨推盤盤送至揀貨點之最遲時刻，$w \in Integer^+$} \\
\end{array}\\
$$

In [1]:
from gurobipy import *

In [2]:
def print_dict(dict):
    print('{')
    for k, v in dict.items():
        print(f'\t{k}: {v}')
    print('}')
    
def print_2dlist(list):
    print('[')
    for row in list:
        print(f'\t{row}')
    print(']')

In [7]:
# import preprocess_spa
%run preprocess_spa.ipynb
########## 改版 ##########
size = (3,3)
n_sp = 2
n_tar = 3
id = 4
##### time limit #####
# size = (6,6)
# n_sp = 18
# n_tar = 18
# id = 2
# size = (3,3)
# n_sp = 1
# n_tar = 6
# id = 5
map_grid, map_start, map_end, T_ub, T, N, R, A, In_ri, Out_ri, P_ij, Ci, Hi, Vi, I, P, S_ri, F_ri, M, epsilon, Num = preprocess(size, n_sp, n_tar, id)

# ########## Sets ##########
# print(T)
# print(N)
# print(A)
# print(R)
# print_dict(Out_ri)
# print_dict(In_ri)
# print_dict(P_ij)
# print_dict(Ci)
# print_dict(Hi)
# print_dict(Vi)
# print(I)
# print(P)

# ########## Parameters ##########
# print(M)
# print(epsilon)
# print_dict(S_ri)
# print_dict(F_ri)
print_2dlist(map_start)
print_2dlist(map_end)
print(Num)

[
	[2, -1, 0]
	[0, -1, -1]
	[-1, 1, 3]
]
[
	[3, -1, 2]
	[1, -1, -1]
	[-1, 0, 0]
]
[2, 1, 1, 1]


### **數學模式**
---
$$
\begin{array}{lll}
   Minimize & w + \epsilon *\displaystyle\sum_{t\in \{0\} \cup T} \displaystyle\sum_{(i,j) \in A, i \neq j} |P_{ij}| * f^{t0}_{ij}\\
   \\
   Subject \quad To & \\
   \\
   \textbf{流量守恆}\\
   \\
   選一條出去(或沒有) & \displaystyle\sum_{(i,j) \in \delta^{out}_{ri}} f_{ij}^{0r} = S^r_i & \forall  r \in R \\
   各類推盤守恆 & \displaystyle\sum_{i \in N} \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{|T|r} = Num^r & \forall r \in R \\
   前時刻進=現時刻出 & \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{(t-1)r} = \displaystyle\sum_{(i,j) \in \delta^{out}_{ri}} f_{ij}^{tr} & \forall i \in N ; t \in T ; r \in R \\
    \\
    \textbf{空格限制} \\
    \\
    同時同格點至多一位 & \displaystyle\sum_{r \in R} \displaystyle\sum_{(j,i) \in \delta^{in}_{ri}} f_{ji}^{tr} \leq 1 & \forall i \in N ; t \in \{0\} \cup T \\
    不允許空格區塊位移(移入移出擇一) & \displaystyle\sum_{(j,i) \in \delta^{in}_{0i}, i \neq j} f_{ji}^{t0} + \displaystyle\sum_{(i,j) \in \delta^{out}_{0i} } f_{ij}^{t0} \leq 1 & \forall i \in N ; t \in \{0\} \cup T \\
    出貨推盤移動路徑上要有空格 & \displaystyle\sum_{r \in R \backslash \{0\}} f_{ji}^{tr} \leq \displaystyle\sum_{(k,i) \in \delta^{in}_{0i}} \displaystyle\sum_{(k,l) \in \delta^{out}_{0k}, (i,j) \subset P_{kl}, k < min(i,l) } f_{kl}^{t0} & \forall i \in N ; (i,j) \in \delta^{out}_{r' i} , r' \in R \backslash \{0\} , i>j ; t\in \{0\} \cup T \\
    出貨推盤移動路徑上要有空格 & \displaystyle\sum_{r \in R \backslash \{0\}} f_{ji}^{tr} \leq \displaystyle\sum_{(k,i) \in \delta^{in}_{0i}} \displaystyle\sum_{(k,l) \in \delta^{out}_{0k}, (i,j) \subset P_{kl}, k > max(i,l) } f_{kl}^{t0} & \forall i \in N ; (i,j) \in \delta^{out}_{r' i} , r' \in R \backslash \{0\} , i>j ; t\in \{0\} \cup T \\
    空格移動路徑上的出貨推盤必定動 & \displaystyle\sum_{r \in R \backslash \{0\}} \displaystyle\sum_{k \in P_{ij}, k \neq j} f^{tr}_{kk} \leq M * (1 - f^{t0}_{ij}) & \forall i \in N ; (i,j) \in \delta^{out}_{0i} ; t \in \{0\} \cup T \\
    空格移動路徑上不存在其他空格重疊 & \displaystyle\sum_{k \in P_{ij}, k \neq j} \displaystyle\sum_{(k,l) \in \delta^{out}_{0k}, l \notin \{i,j\}} f^{t0}_{kl} + \displaystyle\sum_{k \in P_{ij}, k \neq j} \displaystyle\sum_{(l,k) \in \delta^{in}_{0k}, l \notin \{i,j,k\}} f^{t0}_{lk}\leq M * (1 - f^{t0}_{ij}) & \forall i \in N ; (i,j) \in \delta^{out}_{0i} ; t \in \{0\} \cup T \\
    空格不會垂直跨越 & \displaystyle\sum_{(k,l) \in C_i} f^{t0}_{kl} \leq 1 & \forall i \in N ; t \in \{0\} \cup T
    \\
    \textbf{轉向限制} \\
    \\
    實體貨物水平轉垂直 & \displaystyle\sum_{(i,j_1) \in H_i} \displaystyle\sum_{(j_2,i) \in V_i} (f_{ij_1}^{(t-1)0} + f_{j_2i}^{t0}) \leq 1 & \forall i \in N ; t \in T \\
    實體貨物垂直轉水平 & \displaystyle\sum_{(i,j_1) \in V_i} \displaystyle\sum_{(j_2,i) \in H_i} (f_{ij_1}^{(t-1)0} + f_{j_2i}^{t0}) \leq 1 & \forall i \in N ; t \in T \\
    \\
    \textbf{時間限制} \\
    \\
    紀錄抵達終點時間 & \displaystyle\sum_{(P_r, j) \in A} f^{tr}_{P_rj} \geq x^t_r & \forall t \in \{0\} \cup T ; r \in R \backslash \{0\} \\
    至少經過終點一次 & \displaystyle\sum_{t \in \{0\} \cup T} x^t_r \geq 1 & \forall r \in R \backslash \{0\} \\
    最遲抵達終點時刻 & \displaystyle\sum_{t \in \{0\} \cup T} (t * x^t_r) \leq w & \forall r \in R \backslash \{0\} \\
    \\
    Bound & \\
          & f^{tr}_{ij} \in \{0,1\} & \forall r \in R ; (i,j) \in A ; t \in \{0\} \cup T \\
          & x^t_r \in \{0,1\} & \forall r \in R \backslash \{0\} ; t \in \{0\} \cup T\\
          & w \in Integer^+ \\
    \\
\end{array}\\            
$$

In [9]:
model = Model("Puzzle-Based Reshuffling SPA")

T_ub = 30
T = [i+1 for i in range(T_ub)]

f = {}
x = {}
w = 0
for t in [0]+T:
    for r in R:
        x[t,r] = model.addVar(vtype = "B", name = "x(%d,%d)" % (t,r))
        for i in N:
            for j in N:
                f[t,r,i,j] = model.addVar(vtype = "B", name = "f(%d,%d,%d,%d)" % (t,r,i,j))

w = model.addVar(vtype = "I", name = "w")

for r in R:
    for i in N:
        # 時間 0，所有貨物 r 從起點出發遵守流量守恆
        model.addConstr(quicksum(f[0, r, i, j[1]] for j in Out_ri[r][i]) == S_ri[r][i], name = f"Start_R{r}_N{i}")
        # 所有相鄰時刻，同個點同個貨物要有一進一出
        for t in T:
            model.addConstr(quicksum(f[t-1, r, j[1], i] for j in In_ri[r][i]) == quicksum(f[t, r, i, j[1]] for j in Out_ri[r][i]),
                            name = f"Flow_balance_T{t}_R{r}_N{i}")
            
# 時間 T，所有推盤類別 r 抵達終點，且同類推盤總和遵守流量守恆
for r in R:
    model.addConstr(quicksum(f[T_ub, r, j[1], i] for i in N for j in In_ri[r][i]) == Num[r], name = f"End_R{r}")
            
for t in [0]+T:
    for i in N:
        # 一格一托盤
        model.addConstr(quicksum(f[t, r, j[1], i] for j in In_ri[r][i] for r in R) <= 1, 
                        name = f"Grid_{i}_Limit_in_time_{t}")
        # 不允許空格區塊位移
        model.addConstr(quicksum(f[t, 0, j[1], i] for j in In_ri[0][i] if j[1] != i) + 
                        quicksum(f[t, 0, i, j[1]] for j in Out_ri[0][i] if j[1] != i) <= 1, 
                        name = f"No_block_move_{i}_Limit_in_time_{t}")
        
        # 出貨推盤移動路徑上要有空格
        for j in Out_ri[1][i]:    # 不能是 [0][i]，我們要找實體路徑
            if i > j[1]:
                model.addConstr(quicksum(f[t, r, i, j[1]] for r in R[1:]) <= \
                                quicksum(f[t, 0, k[1], l[1]] for k in In_ri[0][i] for l in Out_ri[0][k[1]] \
                                         if i in P_ij[k[1]][l[1]] and j[1] in P_ij[l[1]][k[1]] and k[1] < l[1] and k[1] < i
                                        ), 
                                name = f"Space_On_Route_T{t}_({i},{j[1]})")
            if i < j[1]:
                model.addConstr(quicksum(f[t, r, i, j[1]] for r in R[1:]) <= \
                                quicksum(f[t, 0, k[1], l[1]] for k in In_ri[0][i] for l in Out_ri[0][k[1]] \
                                         if i in P_ij[k[1]][l[1]] and j[1] in P_ij[l[1]][k[1]] and k[1] > l[1] and k[1] > i
                                        ), 
                                name = f"Space_On_Route_T{t}_({i},{j[1]})")

        # 空格移動路徑上的出貨推盤必定動
        for j in Out_ri[0][i]:
            if i != j[1]:
                if len(P_ij[i][j[1]]) != 1:
                    model.addConstr(quicksum(f[t, r, k, k] for k in P_ij[i][j[1]] if k != j[1] for r in R[1:]) 
                                    <= M * (1 - f[t, 0, i, j[1]]),
                                    name = f"Must_move_target_T{t}_({i},{j[1]})")
        # 空格移動路徑上不存在其他空格重疊
                    model.addConstr(quicksum(f[t, 0, k, l[1]] for k in P_ij[i][j[1]] if k != j[1] for l in Out_ri[0][k] if l[1] not in {i, j[1]}) + \
                                    quicksum(f[t, 0, l[1], k] for k in P_ij[i][j[1]] if k != j[1] for l in Out_ri[0][k] if l[1] not in {i, j[1], k}) \
                                    <= M * (1 - f[t, 0, i, j[1]]),
                                    name = f"No_cross_space_T{t}_({i},{j[1]})")

        # 空格不會垂直跨越
        if len(Ci[i]["Hori"]) > 0 and len(Ci[i]["Verti"]) > 0:
            model.addConstr(quicksum(f[t, 0, k, l] for (k,l) in Ci[i]["Hori"]) + quicksum(f[t, 0, k, l] for (k,l) in Ci[i]["Verti"]) <= 1,
                            name = f"No_cross_T{t}_N{i}")

# 轉向限制
for t in T:
    for i in N:
        model.addConstr(quicksum(f[t-1, 0, i, j1] for j1 in Hi[i]) + \
                        quicksum(f[t, 0, j2, i] for j2 in Vi[i]) <= 1,
                        name = f"H{i}_turn_V{i}")
        model.addConstr(quicksum(f[t-1, 0, i, j1] for j1 in Vi[i]) + \
                        quicksum(f[t, 0, j2, i] for j2 in Hi[i]) <= 1,
                        name = f"V{i}_turn_H{i}")


# # 至少經過終點一次
# for r in R[1:]:
#     model.addConstr(quicksum(f[t, r, endpos[0], endpos[1]] for t in [0]+T for endpos in A if endpos[1] == P[Num[0]+r-1][1]) >= Num[r], 
#                     name = f"AtLeast_R{r}_N{P[Num[0]+r-1][1]}")
# # 最遲時刻
# for r in R[1:]:
#     for t in [0]+T:
#         for endpos in A:
#             if endpos[1] == P[Num[0]+r-1][1]:
#                 model.addConstr(w + M * (1 - f[t,r,endpos[1],endpos[0]]) >= t, 
#                                 name = f"Finish_R{r}_N{endpos[1]}_T{t}")

# 紀錄推盤編號及抵達時間
for t in [0]+T:
    for r in R[1:]:
        for endpos in P:
            if endpos[0] == r:
                model.addConstr(quicksum(f[t, r, endpos[1], j[1]] for j in Out_ri[r][endpos[1]]) >= x[t,r], name = f"Finish_T{t}_R{r}")
# 至少經過一次終點
for r in R[1:]:
    model.addConstr(quicksum(x[t,r] for t in [0]+T) >= 1,name = f"Finish_Position_R{r}")
# 最遲時刻
for r in R[1:]:
    model.addConstr(quicksum(t*x[t,r] for t in [0]+T) <= w, name = f"Time_Limit_R{r}")   



is_running = False
timelimit = 350
model.Params.LogtoConsole = is_running  # 是否列出軟體求解過程
model.Params.timeLimit = timelimit      # 求解時限
model.setObjective(w + epsilon * quicksum(len(P_ij[i][j[1]]) * f[t, 0, i, j[1]] for t in [0]+T for i in N for j in In_ri[0][i]) , GRB.MINIMIZE)
model.optimize()

def printModel(map_start):
    print("running time = ", model.Runtime)
    print("optimal value = ", model.objVal)
    map = []
    for row in map_start:
        map.append(row.copy())
    for t in [0]+T:
        print('-' * 40)
        print(f"time period {t}")
        for row in map:
            print('|', end = ' ')
            for col in row:
                if col == -1: col = ' X'
                print(f'{col:2}', end = ' ')
            print('|')

        map = []
        for i in range(size[0]):
            row = [' X'] * size[1]
            map.append(row)
        print()
        for r in R:
            for (i,j) in A:
                var_name = f"f({t},{r},{i},{j})"
                var = model.getVarByName(var_name)
                EPS = 1.e-6
                if var.X > EPS:
                    if i != j:
                        print(var_name, var.X)
                    if r == 0:
                        map[(j-1)//size[1]][(j-1)%size[1]] = 0
                    else:
                        map[(j-1)//size[1]][(j-1)%size[1]] = r

if model.status == GRB.OPTIMAL:
    model.write("model.lp")      # 可行約束條件
    # Print Optimal value and solution
    printModel(map_start)
        
elif model.status == GRB.TIME_LIMIT:    
    print("Time out, cannot get a optimal solution")
    
elif model.status == GRB.INFEASIBLE:
    model.computeIIS()
    model.write("model.ilp")   # 寫出不可行的約束條件
    print("Infeasible, no solution found.")



running time =  25.26200008392334
optimal value =  8.016
----------------------------------------
time period 0
|  2  X  0 |
|  0  X  X |
|  X  1  3 |

f(0,0,4,5) 1.0
----------------------------------------
time period 1
|  2  X  0 |
|  X  0  X |
|  X  1  3 |

f(1,0,5,8) 1.0
f(1,0,3,1) 1.0
f(1,1,8,5) 1.0
f(1,2,1,2) 1.0
----------------------------------------
time period 2
|  0  2  X |
|  X  1  X |
|  X  0  3 |

f(2,0,1,4) 1.0
f(2,0,8,9) 1.0
f(2,3,9,8) 1.0
----------------------------------------
time period 3
|  X  2  X |
|  0  1  X |
|  X  3  0 |

f(3,0,4,5) 1.0
f(3,0,9,3) 1.0
f(3,1,5,4) 1.0
----------------------------------------
time period 4
|  X  2  0 |
|  1  0  X |
|  X  3  X |

f(4,0,3,2) 1.0
f(4,0,5,8) 1.0
f(4,2,2,3) 1.0
f(4,3,8,5) 1.0
----------------------------------------
time period 5
|  X  0  2 |
|  1  3  X |
|  X  0  X |

f(5,0,2,5) 1.0
f(5,0,8,7) 1.0
f(5,3,5,2) 1.0
----------------------------------------
time period 6
|  X  3  2 |
|  1  0  X |
|  0  X  X |

f(6,0,7,